## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
# installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.45 --force-reinstall --upgrade --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 307.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 297.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 281.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 359.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 300.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= 

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
!pip install -U transformers huggingface_hub accelerate

In [ ]:
import transformers, huggingface_hub
print(transformers.__version__)
print(huggingface_hub.__version__)

4.57.1
0.36.0


In [2]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## **Loading the Large Language Model**

In [3]:
## Model configuration
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [4]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=256,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [5]:
# function to generate, process, and return the response from the LLM
def generate_llama_response(user_prompt):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

## Question Answering using LLM

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9054.86 ms
llama_print_timings:      sample time =     139.43 ms /   256 runs   (    0.54 ms per token,  1836.01 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   85291.84 ms /   256 runs   (  333.17 ms per token,     3.00 tokens per second)
llama_print_timings:       total time =   86376.76 ms /   257 tokens


 The management of sepsis in a critical care unit (CCU) follows a standardized protocol that includes early recognition, aggressive fluid resuscitation, antibiotic therapy, and close monitoring. Here are the key components of the protocol:

1. Early recognition: CCU staff should be vigilant in identifying patients at risk for sepsis, such as those with suspected or confirmed infections, severe injuries, or organ dysfunction. Early recognition can help ensure timely intervention and improve outcomes.
2. Assessment: A thorough assessment should be performed, including vital signs, clinical examination, and laboratory tests, such as blood cultures, complete blood counts, and serum biomarkers like procalcitonin and lactate. This information helps identify the source and severity of sepsis and guides treatment decisions.
3. Fluid resuscitation: Sepsis patients often have hypovolemia or hypoperfusion, which can worsen organ dysfunction. CCU staff should initiate fluid resuscitation with crys

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [6]:
#user_prompt = ""
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = generate_llama_response(user_prompt)

print(response)


llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     144.75 ms /   256 runs   (    0.57 ms per token,  1768.59 tokens per second)
llama_print_timings: prompt eval time =    3054.30 ms /    67 tokens (   45.59 ms per token,    21.94 tokens per second)
llama_print_timings:        eval time =   79847.67 ms /   255 runs   (  313.13 ms per token,     3.19 tokens per second)
llama_print_timings:       total time =   84037.30 ms /   322 tokens


 Sure, I'd be happy to help! Appendicitis is a medical emergency that requires prompt treatment. Here are some common symptoms of appendicitis:

1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen. The pain may be constant or come and go.
2. Nausea and vomiting.
3. Loss of appetite.
4. Fever.
5. Abdominal tenderness and guarding (muscle tension) in the affected area.
6. Abdominal swelling.
7. Inability to pass gas.
8. Inability to move the bowels.

If you suspect that you or someone else may have appendicitis, it is important to seek medical attention immediately. Appendicitis cannot be cured with medication alone; instead, surgery is required to remove the inflamed appendix. The most common surgical procedure used to treat appendicitis is an appendectomy.

There are two types of appendectomies: open and laparoscopic. An open appendectomy involves a larger


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
#user_prompt = ""
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9054.86 ms
llama_print_timings:      sample time =     153.74 ms /   256 runs   (    0.60 ms per token,  1665.13 tokens per second)
llama_print_timings: prompt eval time =    1027.19 ms /    69 tokens (   14.89 ms per token,    67.17 tokens per second)
llama_print_timings:        eval time =   82930.16 ms /   255 runs   (  325.22 ms per token,     3.07 tokens per second)
llama_print_timings:       total time =   85086.57 ms /   324 tokens


 Sudden patchy hair loss, also known as alopecia areata, can be a distressing condition that affects millions of people worldwide. While there is no definitive cure for this condition, there are several effective treatment options available that can help promote hair growth and reduce inflammation. Here are some of the most common treatment options for sudden patchy hair loss:
    1. Topical corticosteroids: Over-the-counter (OTC) creams or prescription-strength corticosteroids can help reduce inflammation and promote hair growth. These medications can be applied directly to the affected area twice daily.
    2. Minoxidil: Minoxidil is a solution that is applied directly to the affected area to stimulate hair growth. It is available OTC and can be used in combination with corticosteroids.
    3. Anthralin: Anthralin is a medication that can help reduce inflammation and promote hair growth. It is available in both OTC and prescription strengths.
    4. Phototherapy: Exposure to specific

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
#user_prompt = ""
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9054.86 ms
llama_print_timings:      sample time =     134.19 ms /   231 runs   (    0.58 ms per token,  1721.47 tokens per second)
llama_print_timings: prompt eval time =     962.14 ms /    63 tokens (   15.27 ms per token,    65.48 tokens per second)
llama_print_timings:        eval time =   75578.18 ms /   230 runs   (  328.60 ms per token,     3.04 tokens per second)
llama_print_timings:       total time =   77563.60 ms /   293 tokens


 Treatment options for a person with a physical injury to brain tissue depend on several factors such as the location and severity of the injury. However, here are some common treatment recommendations that may be considered: 
    1. Medications: To manage symptoms such as pain, inflammation, and anxiety. 
    2. Rehabilitation therapy: To improve cognitive and motor functions and regain independence. 
    3. Surgery: To relieve pressure on the affected area and repair damaged blood vessels. 
    4. Lifestyle modifications: To promote healing and reduce stress. 
    5. Neuropsychological testing: To assess cognitive and emotional functions and develop an appropriate treatment plan. 
    6. Alternative therapies: Such as acupuncture and massage may also be considered. 
     It is important to note that each individual's treatment plan is unique and may involve a combination of these options. It is essential to work with a healthcare provider to determine the best course of treatment.


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
#user_prompt = ""
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9054.86 ms
llama_print_timings:      sample time =     151.53 ms /   256 runs   (    0.59 ms per token,  1689.46 tokens per second)
llama_print_timings: prompt eval time =    1032.52 ms /    69 tokens (   14.96 ms per token,    66.83 tokens per second)
llama_print_timings:        eval time =   84344.45 ms /   255 runs   (  330.76 ms per token,     3.02 tokens per second)
llama_print_timings:       total time =   86525.86 ms /   324 tokens


 Sure, I'd be happy to help! If someone has fractured their leg while hiking, it is essential to take immediate action to provide proper care and prevent further complications. Here are some necessary precautions and treatment steps that should be taken:

1. Stop activity: The first step is to stop any further activity that may exacerbate the injury. This includes halting any further hiking or walking.
2. Assess the injury: Assess the severity of the injury by checking for any deformities, swelling, or open wounds. If the fracture is displaced or open, do not move the affected limb.
3. Immobilize the affected limb: Use a splint or brace to immobilize the affected limb. This will help reduce pain and prevent further damage.
4. Elevate the affected limb: Elevate the affected limb above heart level to reduce swelling.
5. Apply ice: Apply ice packs to reduce pain and swelling. Wrap ice packs in a towel or cloth to avoid direct contact with the skin.
6. Monitor vital signs: Monitor


## Question Answering using LLM with Prompt Engineering

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [7]:
user_prompt = '''You are a highly reliable and clinically accurate medical assistant.
Provide a medically correct, evidence-based answer to the question provided.
Use structured reasoning, avoid hallucinations, avoid assumptions, and follow
established medical guidelines.
### Requirements:
- Provide a precise, clinically accurate explanation.
- Include symptoms, causes, diagnosis, treatment, and follow-up (when relevant).
- Avoid giving unverifiable facts.
- Keep the answer clear, structured, and evidence-informed

### Question:
What is the protocol for managing sepsis in a critical care unit?

### Format:
1. Short Direct Answer
2. Key Details (Symptoms / Causes / Mechanism)
3. Step-by-Step Management or Treatment
'''
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     145.75 ms /   256 runs   (    0.57 ms per token,  1756.49 tokens per second)
llama_print_timings: prompt eval time =    1515.21 ms /   236 tokens (    6.42 ms per token,   155.75 tokens per second)
llama_print_timings:        eval time =   83604.27 ms /   255 runs   (  327.86 ms per token,     3.05 tokens per second)
llama_print_timings:       total time =   86266.94 ms /   491 tokens


1. Short Direct Answer: Sepsis management in critical care units involves early recognition, prompt administration of antibiotics and fluid resuscitation, close monitoring of vital signs and organ dysfunction, and timely interventions to address any complications or progression of the disease.

    2. Key Details (Symptoms / Causes / Mechanism):
        A. Symptoms: Fever, tachycardia, tachypnea, confusion, and organ dysfunction (e.g., renal impairment, hepatic dysfunction)
        B. Causes: Bacterial infections (e.g., pneumonia, urinary tract infections), viral infections (e.g., influenza), fungal infections (e.g., candidemia), or parasitic infections (e.g., malaria) that spread through the bloodstream and cause systemic inflammation
        C. Mechanism: Sepsis is caused by an exaggerated immune response to an infection, leading to uncontrolled inflammation and tissue damage

    3. Step


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [8]:
user_prompt = '''You are a highly reliable and clinically accurate medical assistant.
Provide a medically correct, evidence-based answer to the question provided.
Use structured reasoning, avoid hallucinations, avoid assumptions, and follow
established medical guidelines.
### Requirements:
- Provide a precise, clinically accurate explanation.
- Include symptoms, causes, diagnosis, treatment, and follow-up (when relevant).
- Avoid giving unverifiable facts.
- Keep the answer clear, structured, and evidence-informed

### Question:
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

### Format:
1. Short Direct Answer
2. Key Details (Symptoms / Causes / Mechanism)
3. Step-by-Step Management or Treatment
'''
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     143.99 ms /   256 runs   (    0.56 ms per token,  1777.94 tokens per second)
llama_print_timings: prompt eval time =    1043.87 ms /   121 tokens (    8.63 ms per token,   115.92 tokens per second)
llama_print_timings:        eval time =   82942.56 ms /   255 runs   (  325.26 ms per token,     3.07 tokens per second)
llama_print_timings:       total time =   85096.09 ms /   376 tokens


1. Short Direct Answer: Appendicitis is a medical emergency that requires prompt surgical intervention; there is no medication cure for appendicitis.
    2. Key Details (Symptoms / Causes / Mechanism):
        Symptoms: Sudden onset of severe pain in the lower right abdomen (often migrating to the lower back), nausea, vomiting, fever, loss of appetite, and abdominal tenderness.
        Causes: Obstruction of the appendix by feces, foreign bodies, or inflamed appendix tissue (appendicitis).
        Mechanism: Inflammation of the appendix leads to swelling and obstruction of blood flow, causing tissue death (necrosis) and potentially life-threatening complications if left untreated.
    3. Step-by-Step Management or Treatment:
        Diagnosis: Physical examination, medical history, laboratory tests (complete blood count, blood culture), and imaging studies (plain X-rays or CT scans) to confirm appendicitis and rule out other conditions.



### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [9]:
user_prompt = '''You are a highly reliable and clinically accurate medical assistant.
Provide a medically correct, evidence-based answer to the question provided.
Use structured reasoning, avoid hallucinations, avoid assumptions, and follow
established medical guidelines.
### Requirements:
- Provide a precise, clinically accurate explanation.
- Include symptoms, causes, diagnosis, treatment, and follow-up (when relevant).
- Avoid giving unverifiable facts.
- Keep the answer clear, structured, and evidence-informed

### Question:
What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

### Format:
1. Short Direct Answer
2. Key Details (Symptoms / Causes / Mechanism)
3. Step-by-Step Management or Treatment
'''
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     143.52 ms /   256 runs   (    0.56 ms per token,  1783.70 tokens per second)
llama_print_timings: prompt eval time =    1049.34 ms /   125 tokens (    8.39 ms per token,   119.12 tokens per second)
llama_print_timings:        eval time =   83001.62 ms /   255 runs   (  325.50 ms per token,     3.07 tokens per second)
llama_print_timings:       total time =   85168.97 ms /   380 tokens


1. Short Direct Answer: Sudden patchy hair loss with localized bald spots on the scalp can be caused by various factors such as alopecia areata, fungal infections, or bacterial infections. Effective treatments include topical or oral medications, corticosteroid injections, or immunotherapy.
    2. Key Details (Symptoms / Causes / Mechanism):
        Symptoms: Sudden patchy hair loss with well-defined bald spots on the scalp, itching or burning sensation on the affected area, and redness or scaling of the skin.
        Causes: Alopecia areata (an autoimmune condition), fungal infections (such as ringworm), or bacterial infections (such as folliculitis). The exact cause may not be identified in some cases.
        Mechanism: The immune system mistakenly attacks healthy hair follicles leading to hair loss, while fungal or bacterial infections can cause inflammation and damage to the hair follicles.
    3. Step-by-Step Management or Treat


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [10]:
user_prompt = '''You are a highly reliable and clinically accurate medical assistant.
Provide a medically correct, evidence-based answer to the question provided.
Use structured reasoning, avoid hallucinations, avoid assumptions, and follow
established medical guidelines.
### Requirements:
- Provide a precise, clinically accurate explanation.
- Include symptoms, causes, diagnosis, treatment, and follow-up (when relevant).
- Avoid giving unverifiable facts.
- Keep the answer clear, structured, and evidence-informed

### Question:
What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

### Format:
1. Short Direct Answer
2. Key Details (Symptoms / Causes / Mechanism)
3. Step-by-Step Management or Treatment
'''
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     146.54 ms /   256 runs   (    0.57 ms per token,  1747.01 tokens per second)
llama_print_timings: prompt eval time =    1058.32 ms /   119 tokens (    8.89 ms per token,   112.44 tokens per second)
llama_print_timings:        eval time =   80721.82 ms /   255 runs   (  316.56 ms per token,     3.16 tokens per second)
llama_print_timings:       total time =   82902.61 ms /   374 tokens


1. Short Direct Answer: The treatment for physical injury to brain tissue depends on the severity of the injury, location of injury, age of patient, and other underlying health conditions. Treatment options may include medication, surgery, rehabilitation therapy, or a combination of these interventions.
    2. Key Details (Symptoms / Causes / Mechanism):
        Symptoms: Headache, dizziness, confusion, weakness or numbness in limbs, difficulty with speech or language, memory loss, changes in behavior or mood, seizures, or loss of consciousness.
        Causes: Traumatic brain injury (TBI) caused by a blow or jolt to the head or body that results in brain tissue damage. Common causes include car accidents, falls, sports injuries, or violence.
        Mechanism: The injury disrupts normal brain function by damaging brain cells or disrupting communication between brain cells, leading to temporary or permanent impairment of brain function.
    3. Step-by-Step Management or Treatment:
    

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [11]:
user_prompt = '''You are a highly reliable and clinically accurate medical assistant.
Provide a medically correct, evidence-based answer to the question provided.
Use structured reasoning, avoid hallucinations, avoid assumptions, and follow
established medical guidelines.
### Requirements:
- Provide a precise, clinically accurate explanation.
- Include symptoms, causes, diagnosis, treatment, and follow-up (when relevant).
- Avoid giving unverifiable facts.
- Keep the answer clear, structured, and evidence-informed

### Question:
What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

### Format:
1. Short Direct Answer
2. Key Details (Symptoms / Causes / Mechanism)
3. Step-by-Step Management or Treatment
'''
response = generate_llama_response(user_prompt)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3054.47 ms
llama_print_timings:      sample time =     145.75 ms /   256 runs   (    0.57 ms per token,  1756.44 tokens per second)
llama_print_timings: prompt eval time =    1061.67 ms /   125 tokens (    8.49 ms per token,   117.74 tokens per second)
llama_print_timings:        eval time =   82059.76 ms /   255 runs   (  321.80 ms per token,     3.11 tokens per second)
llama_print_timings:       total time =   84244.54 ms /   380 tokens


1. Short Direct Answer:
If someone has fractured their leg during a hiking trip, it is essential to seek immediate medical attention to prevent further complications and ensure proper healing.

    2. Key Details (Symptoms / Causes / Mechanism):
Symptoms: Pain in the affected leg, swelling, bruising, difficulty moving the leg, and deformity (if the fracture is displaced). Causes: Trauma from falling or twisting during hiking activities can cause fractures in the leg bones (femur, tibia, fibula). Mechanism: The force of the trauma causes the bone to break, leading to displacement or displacement of the bone fragments.

    3. Step-by-Step Management or Treatment:
Step 1: Initial Assessment - Check for airway, breathing, and circulation (ABCs) and assess the severity of the injury based on the symptoms and mechanism of injury. Step 2: Immobilization - Use a splint or cast to immobilize the affected leg and prevent further movement or displacement of the fract


## Data Preparation for RAG

In [12]:
# For installing the libraries & downloading models from HF Hub
#!pip install -q tiktoken==0.6.0 pypdf==4.0.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2

!pip install -q tiktoken pypdf langchain langchain-community chromadb sentence-transformers numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.4 MB/s eta 0

In [13]:
import json
import tiktoken

import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.vectorstores import Chroma

from google.colab import userdata, drive

In [14]:
import warnings
warnings.filterwarnings('ignore')

### Loading the Data

In [16]:
# uncomment and run the below code snippets if the dataset is present in the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
pdf_file = "/content/drive/MyDrive/Python Course/NLPWithGenAI/Project-MedicalAssistant/medical_diagnosis_manual.pdf"

In [18]:
pdf_loader = PyPDFLoader(pdf_file)

### Data Overview

#### Checking the first 5 pages

In [22]:
# Get all pages as LangChain Documents
docs = pdf_loader.load()

first_5_pages = docs[:5]   # first 5
for i, d in enumerate(first_5_pages, 1):
    print(f"\n--- Page {i} ---")
    print(d.page_content[:400])  # Show first 400 chars


--- Page 1 ---
gvishweshwar83@gmail.com
P4ACI6Y9ST
This file is meant for personal use by gvishweshwar83@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

--- Page 2 ---
gvishweshwar83@gmail.com
P4ACI6Y9ST
This file is meant for personal use by gvishweshwar83@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

--- Page 3 ---
Table of Contents
1
Front  
  ................................................................................................................................................................................................................
1
Cover  
  .....................................................................................................................................................

--- Page 4 ---
491 Chapter 44. Foot & Ankle Disorders    ...........................................................................................................................

#### Checking the number of pages

In [28]:
from langchain_community.document_loaders import PyPDFLoader

#loader = PyPDFLoader(pdf_file)
documents = pdf_loader.load_and_split()
print(f"Loaded {len(documents)} pages.")

Loaded 5142 pages.


### Data Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [ ]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
i = 5
document_chunks[i]

Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'moddate': '2025-10-31T11:55:58+00:00', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'source': '/content/drive/MyDrive/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'page': 3, 'page_label': 'iv'}, page_content='Chapter 68. Retinal Disorders  \n  .................................................................................................................................................\n731\nChapter 69. Optic Nerve Disorders  \n  ......................................................................................................................................\n737\nChapter 70. Orbital Diseases  \n  ..................................................................................................................................................\n742\n7 - Dermatologic Disorders  \n  ............

In [ ]:
document_chunks[i+1]

Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'moddate': '2025-10-31T11:55:58+00:00', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'source': '/content/drive/MyDrive/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'page': 3, 'page_label': 'iv'}, page_content='P4ACI6Y9ST\nThis file is meant for personal use by gvishweshwar83@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.')

### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Vector Database

In [ ]:
report = 'Article_QnA'
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    collection_name=report
)

### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 6}
)

### System and User Prompt Template

In [ ]:
qna_system_message = """
You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context.

"""

In [ ]:
qna_user_message_template = """
Conider the following ###Context and ###Question
###Context
{context}

###Question
{question}
"""

### Response Function

In [ ]:
def RAG(user_input , llm):
    """
    Args:
        user_input: Takes a user input for which the response should be retrieved from the vectorDB.
        llm: The LLM to be used for generating the response
    Returns:
        The generated response based on the user query and the context from the knowledge base
    """
    relevant_document_chunks = retriever.get_relevant_documents(user_input)
    context_list = [d.page_content.replace("\t", " ") for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)



    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""


    # Quering an LLM
    try:
        response = llm(
                prompt=prompt,
                max_tokens=500,
                temperature=0.4,
                top_p=0.95,
                repeat_penalty=1.2,
                top_k=25,
                stop=['INST'],
                echo=False
                )

        prediction =  response["choices"][0]["text"]

    except Exception as e:
        prediction = f'Sorry, I encountered the following error: \n {e}'

    return  prediction

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(RAG("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm))
#print(generate_rag_response("What is the protocol for managing sepsis in a critical care unit?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     272.71 ms /   463 runs   (    0.59 ms per token,  1697.77 tokens per second)
llama_print_timings: prompt eval time =   31550.82 ms /  3068 tokens (   10.28 ms per token,    97.24 tokens per second)
llama_print_timings:        eval time =  262180.41 ms /   462 runs   (  567.49 ms per token,     1.76 tokens per second)
llama_print_timings:       total time =  296588.63 ms /  3530 tokens


  Based on the provided Context, I can provide an answer to your question as follows:

According to the Context, the protocol for managing sepsis in a critical care unit includes the following steps:

1. Early recognition and diagnosis of sepsis, which includes measuring temperature, heart rate, respiratory rate, and blood pressure, as well as checking for signs of organ dysfunction (such as oliguria, hypotension, and confusion).
2. Aggressive fluid resuscitation with 0.9% saline to raise CVP or PAOP to target levels (usually 8 mm Hg or 10 cm H2O, respectively).
3. Administration of antibiotics, such as gentamicin or ceftazidime, and addition of vancomycin if resistant staphylococci or enterococci are suspected.
4. Monitoring for signs of organ dysfunction, such as oliguria, hypotension, and confusion, and management of any such complications.
5. Close monitoring of blood glucose levels and management of hyperglycemia or hypoglycemia as needed.
6. Surgical excision of infected or necro

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(RAG("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" , lcpp_llm))

#print(generate_rag_response("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     196.58 ms /   344 runs   (    0.57 ms per token,  1749.93 tokens per second)
llama_print_timings: prompt eval time =   31725.55 ms /  3386 tokens (    9.37 ms per token,   106.73 tokens per second)
llama_print_timings:        eval time =  176108.12 ms /   343 runs   (  513.43 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  209790.73 ms /  3729 tokens


  ###Context
We can infer from the provided context that we are dealing with appendicitis and its symptoms, possible treatments, and surgical procedures. The text discusses the classic symptoms, signs, diagnosis, and treatment options for appendicitis, as well as variations in symptoms and signs among different age groups and pregnant women. The text also mentions appendicitis as a clinical diagnosis and emphasizes the importance of early surgery for effective treatment and prevention of complications.

###Question
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Based on the provided context, here's my answer:

Common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, as well as pain that shifts to the right lower quadrant after a few hours and tenderness in the right lower quadrant at McBurney's point. However, these classi

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
#print(RAG("" , lcpp_llm))
print(RAG("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      70.74 ms /   123 runs   (    0.58 ms per token,  1738.74 tokens per second)
llama_print_timings: prompt eval time =   33745.00 ms /  3446 tokens (    9.79 ms per token,   102.12 tokens per second)
llama_print_timings:        eval time =   64400.92 ms /   122 runs   (  527.88 ms per token,     1.89 tokens per second)
llama_print_timings:       total time =   98861.53 ms /  3568 tokens


  I don't know. Based on the information provided, it appears that sudden patchy hair loss, commonly seen as localized bald spots on the scalp, can be caused by various factors such as alopecia, hormonal imbalances, and autoimmune disorders. However, without more information about your specific situation, it's difficult to provide effective treatments or solutions for addressing this issue. It's best to consult a dermatologist or healthcare professional for proper evaluation and diagnosis, and they can recommend appropriate treatment options based on your individual case.


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
#print(RAG("" , lcpp_llm))
print(RAG("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      61.53 ms /   103 runs   (    0.60 ms per token,  1674.01 tokens per second)
llama_print_timings: prompt eval time =   23309.31 ms /  2552 tokens (    9.13 ms per token,   109.48 tokens per second)
llama_print_timings:        eval time =   51158.18 ms /   102 runs   (  501.55 ms per token,     1.99 tokens per second)
llama_print_timings:       total time =   75080.10 ms /  2654 tokens


  Based on the information provided in the given context, there are no direct mentions of treatments for brain injury. However, it is stated that supportive care should include preventing systemic complications, providing good nutrition, preventing pressure ulcers, and providing physical therapy to prevent limb contractures. Additionally, it is recommended that rehabilitation services should be planned early, and patients should be evaluated and reevaluated regularly to prioritize treatment and maximize functional recovery.


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
#print(RAG("" , lcpp_llm))
print(RAG("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     242.20 ms /   411 runs   (    0.59 ms per token,  1696.94 tokens per second)
llama_print_timings: prompt eval time =   31820.42 ms /  3153 tokens (   10.09 ms per token,    99.09 tokens per second)
llama_print_timings:        eval time =  214468.79 ms /   410 runs   (  523.09 ms per token,     1.91 tokens per second)
llama_print_timings:       total time =  248714.21 ms /  3563 tokens


  ###Question: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

###Context: The patient has fractured their leg during a hiking trip and needs proper precautions and treatment steps for their care and recovery. The context includes information about immobilization, splinting, and wound care, as well as potential complications such as infection and compartment syndrome.

Answer: To ensure proper care and recovery for a person who has fractured their leg during a hiking trip, it is necessary to take several precautions and follow established treatment steps. These include:

1. Immobilization: The affected limb should be immobilized using a splint or cast to prevent further injury and promote healing.
2. Wound care: Any open wounds should be cleaned and covered with sterile dressings to prevent infection.
3. Pain management: Analgesics should be administered as 

### Fine-tuning

## Question Answering using RAG - 2 - STRICT (Force model to only answer from retrieved documents)



- This prevents hallucination and forces grounded medical answers.

In [ ]:
def RAG2(user_input, llm):
    relevant_docs = retriever.get_relevant_documents(user_input)
    context = "\n\n".join([d.page_content.replace("\t", " ") for d in relevant_docs])

    prompt = f"""
[INST]
{qna_system_message}

You MUST answer ONLY using the information in the provided context.
If the answer is NOT found in the context, respond exactly with: "The provided context does not contain this information."

### Context:
{context}

### Question:
{user_input}

Provide a concise, medically accurate answer based ONLY on the context. NO assumptions.
[/INST]
"""

    try:
        response = llm(prompt=prompt, max_tokens=400, temperature=0.1, top_p=0.9)
        return response["choices"][0]["text"]
    except Exception as e:
        return f"Error: {e}"


When to use this:

- When you want zero hallucinations
- For medical compliance
- For questions like “What is in the hospital protocol?”

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
#print(RAG2("" , lcpp_llm))
print(RAG2("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     154.00 ms /   242 runs   (    0.64 ms per token,  1571.39 tokens per second)
llama_print_timings: prompt eval time =   35835.89 ms /  3339 tokens (   10.73 ms per token,    93.17 tokens per second)
llama_print_timings:        eval time =  141809.57 ms /   241 runs   (  588.42 ms per token,     1.70 tokens per second)
llama_print_timings:       total time =  179203.53 ms /  3580 tokens


 Based on the provided context, the protocol for managing sepsis in a critical care unit includes:

1. Early recognition of sepsis, including signs of organ dysfunction and failure.
2. Aggressive fluid resuscitation to maintain mean arterial pressure (MAP) of at least 60 mmHg.
3. Administration of vasopressors (such as dopamine or norepinephrine) to maintain MAP if fluid resuscitation is not effective.
4. Antibiotics should be given promptly, and the choice of antibiotics should be guided by the suspected source of infection and the patient's allergies.
5. Monitoring of blood glucose levels and management of hyperglycemia or hypoglycemia as needed.
6. Surgical intervention for source control if appropriate.
7. Close monitoring of organ function and fluid status, and adjustment of therapy as needed.

Note that this answer is based solely on the information provided in the context, and does not include any information from outside the context.


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(RAG2("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     145.21 ms /   243 runs   (    0.60 ms per token,  1673.47 tokens per second)
llama_print_timings: prompt eval time =   30664.88 ms /  3208 tokens (    9.56 ms per token,   104.61 tokens per second)
llama_print_timings:        eval time =  144265.09 ms /   242 runs   (  596.14 ms per token,     1.68 tokens per second)
llama_print_timings:       total time =  176503.76 ms /  3450 tokens


### Context:
• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate.

### Question:
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Answer:
Common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, and pain increases with cough and motion. However, these classic symptoms appear in < 50% of patients, and many variations of symptoms and signs occur. Appendicitis cannot be cured via medicine, and surgical procedure, laparoscopic or open appendectomy, is necessary to treat it.


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(RAG2("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      87.89 ms /   144 runs   (    0.61 ms per token,  1638.34 tokens per second)
llama_print_timings: prompt eval time =   37437.91 ms /  3478 tokens (   10.76 ms per token,    92.90 tokens per second)
llama_print_timings:        eval time =   85426.48 ms /   143 runs   (  597.39 ms per token,     1.67 tokens per second)
llama_print_timings:       total time =  123843.94 ms /  3621 tokens


Based on the provided context, sudden patchy hair loss, commonly seen as localized bald spots on the scalp, can be caused by alopecia areata, telogen effluvium, or anagen effluvium. Treatment options include topical corticosteroids, minoxidil, and immunotherapy. It is essential to consult a dermatologist for a proper diagnosis and treatment plan, as the underlying cause must be determined before initiating therapy. Additionally, it is crucial to avoid any potential triggers, such as stress or excessive styling, and maintain a healthy diet and lifestyle to promote hair growth.


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(RAG2("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      63.11 ms /    96 runs   (    0.66 ms per token,  1521.13 tokens per second)
llama_print_timings: prompt eval time =   29578.63 ms /  2583 tokens (   11.45 ms per token,    87.33 tokens per second)
llama_print_timings:        eval time =   62190.01 ms /    95 runs   (  654.63 ms per token,     1.53 tokens per second)
llama_print_timings:       total time =   92430.32 ms /  2678 tokens


Based on the provided context, treatments recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, include supportive care, rehabilitation services, and neuroprotective agents. The specific treatment plan should be tailored to the individual's needs and should prioritize preventing systemic complications, providing good nutrition, and preventing pressure ulcers.


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(RAG2("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     140.90 ms /   243 runs   (    0.58 ms per token,  1724.58 tokens per second)
llama_print_timings: prompt eval time =   32037.19 ms /  3185 tokens (   10.06 ms per token,    99.42 tokens per second)
llama_print_timings:        eval time =  137274.31 ms /   242 runs   (  567.25 ms per token,     1.76 tokens per second)
llama_print_timings:       total time =  170808.54 ms /  3427 tokens


To ensure proper care and recovery for a person who has fractured their leg during a hiking trip, the following precautions and treatment steps should be taken:

1. Immobilize the affected limb using a splint or cast to prevent further injury and promote healing.
2. Elevate the injured limb above heart level to reduce swelling and pain.
3. Apply ice packs to reduce pain and swelling.
4. Monitor for signs of infection, such as redness, warmth, or increased pain.
5. Provide analgesics as needed to manage pain.
6. Ensure proper hydration and nutrition for optimal healing.
7. Consider early mobilization and range-of-motion exercises to prevent contractures and promote functional recovery.
8. Seek medical attention as soon as possible for proper evaluation and treatment of any complications.

It is essential to consider the severity and type of fracture, as well as any other injuries or complications that may be present, when determining the appropriate course of treatment and care.


## Question Answering using RAG - 3 - EXPANSIVE (Synthesize and add expert clinical reasoning)

- This allows the model to combine context with medical knowledge — for more complete answers.

In [ ]:
def RAG3(user_input, llm):
    relevant_docs = retriever.get_relevant_documents(user_input)
    context = "\n".join([d.page_content.replace("\t", " ") for d in relevant_docs])

    prompt = f"""
[INST]
{qna_system_message}

Your task is to:
1. Extract key facts ONLY from the context.
2. Combine them with standard clinical knowledge.
3. Provide a detailed, structured explanation and treatment plan.

### Context:
{context}

### Question:
{user_input}

Give a detailed clinical answer with:
- Definition
- Key symptoms/findings
- Differential diagnosis (if relevant)
- Step-by-step management protocol
- Follow-up or escalation steps
[/INST]
"""

    try:
        response = llm(prompt=prompt, max_tokens=600, temperature=0.4)
        return response["choices"][0]["text"]
    except Exception as e:
        return f"Error: {e}"


When to use this:

- For full clinical pathways-
- For teaching/explanations
- For expanding beyond context but staying medically correct

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(RAG3("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     347.37 ms /   600 runs   (    0.58 ms per token,  1727.26 tokens per second)
llama_print_timings: prompt eval time =   30856.61 ms /  3175 tokens (    9.72 ms per token,   102.90 tokens per second)
llama_print_timings:        eval time =  320554.42 ms /   599 runs   (  535.15 ms per token,     1.87 tokens per second)
llama_print_timings:       total time =  355191.50 ms /  3774 tokens


 ###Question:
 What is the protocol for managing sepsis in a critical care unit?

Give a detailed clinical answer with:
- Definition
- Key symptoms/findings
- Differential diagnosis (if relevant)
- Step-by-step management protocol
- Follow-up or escalation steps

### Context:
16 - Critical Care Medicine
Chapter 222. Approach to the Critically Ill Patient
Introduction
Critical care medicine specializes in caring for the most seriously ill patients. These patients are best treated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special populations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high nurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring of physiologic parameters.

### Answer:

Definition:
Sepsis is a life-threatening organ dysfunction caused by a dysregulated host response to infection.

Key symptoms/findings:

* Systemic inflammatory response synd

###Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(RAG3("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     338.93 ms /   600 runs   (    0.56 ms per token,  1770.29 tokens per second)
llama_print_timings: prompt eval time =   30602.94 ms /  3232 tokens (    9.47 ms per token,   105.61 tokens per second)
llama_print_timings:        eval time =  304293.10 ms /   599 runs   (  508.00 ms per token,     1.97 tokens per second)
llama_print_timings:       total time =  338650.18 ms /  3831 tokens


 ### Context:

• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate, and it usually lies behind the cecum or the ileum and mesentery of the right colon. A contraindication to appendectomy is inflammatory bowel disease involving the cecum. However, in cases of terminal ileitis and a normal cecum, the appendix should be removed.

### Question:
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Answer:

Definition:
Acute appendicitis is inflammation of the vermiform appendix, typically resulting in abdominal pain, anorexia, and abdominal tenderness.

Key Symptoms/Findings:

1. Epigastric or periumbilical pain (class

###Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(RAG3("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     197.92 ms /   347 runs   (    0.57 ms per token,  1753.27 tokens per second)
llama_print_timings: prompt eval time =   34545.42 ms /  3502 tokens (    9.86 ms per token,   101.37 tokens per second)
llama_print_timings:        eval time =  178997.38 ms /   346 runs   (  517.33 ms per token,     1.93 tokens per second)
llama_print_timings:       total time =  215627.11 ms /  3848 tokens


  ### Context:

Hair loss due to chemotherapy is temporary and is best treated with a wig; when hair regrows, it may be different in color and texture from the original hair. Hair loss due to telogen effluvium or anagen effluvium is usually temporary as well and abates after the precipitating agent is eliminated. The scalp and beard are most frequently affected, but any hairy area may be involved. Alopecia areata is sudden patchy hair loss in people with no obvious skin or systemic disorder. The most frequent adverse effects of minoxidil are mild scalp irritation, allergic contact dermatitis, and increased facial hair. Finasteride inhibits the 5-alpha-reductase enzyme, blocking conversion of testosterone to dihydrotestosterone, and is useful for male-pattern hair loss.

### Question:

What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

Give a detailed

###Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(RAG3("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     342.93 ms /   600 runs   (    0.57 ms per token,  1749.64 tokens per second)
llama_print_timings: prompt eval time =   23897.80 ms /  2607 tokens (    9.17 ms per token,   109.09 tokens per second)
llama_print_timings:        eval time =  296413.02 ms /   599 runs   (  494.85 ms per token,     2.02 tokens per second)
llama_print_timings:       total time =  324059.77 ms /  3206 tokens


 ### Context:
Chapter 324. Traumatic Brain Injury
Introduction
Traumatic brain injury (TBI) is physical injury to brain tissue that temporarily or permanently impairs brain function. Diagnosis is suspected clinically and confirmed by imaging (primarily CT). Initial treatment consists of ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery is often needed in patients with more severe injury to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation.

### Question:
What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

Answer:
Tr

###Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
#print(RAG2("" , lcpp_llm))
print(RAG3("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     341.87 ms /   600 runs   (    0.57 ms per token,  1755.07 tokens per second)
llama_print_timings: prompt eval time =   30902.03 ms /  3209 tokens (    9.63 ms per token,   103.84 tokens per second)
llama_print_timings:        eval time =  307175.88 ms /   599 runs   (  512.81 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  341763.71 ms /  3808 tokens


  ###Context:

Based on the provided context, we can infer that the patient has suffered from a fracture of their leg during a hiking trip. The patient may have symptoms such as pain, swelling, and difficulty moving the affected limb. The goal of treatment is to manage the symptoms, set the bone, and promote healing while minimizing the risk of complications.

###Question:

What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

###Answer:

Step 1: Initial Assessment and Stabilization

* Define the fracture site and assess the extent of soft tissue damage.
* Check for any open wounds or signs of infection.
* Evaluate the patient's neurovascular status, including sensation, motor function, and pulses.
* Stabilize the fracture site using immobilization techniques, such as splinting or casting, to prevent further damage and promote healing.
* Administer pain management

## Question Answering using RAG - 4 - COMPARE (Compare context vs external medical knowledge)





- This helps identify whether the context contradicts or aligns with standard guidelines.

In [ ]:
def RAG4(user_input, llm):
    relevant_docs = retriever.get_relevant_documents(user_input)
    context = "\n".join([d.page_content.replace("\t", " ") for d in relevant_docs])

    prompt = f"""
[INST]
{qna_system_message}

Task:
1. Extract the answer from the context.
2. Provide standard medical guideline information (general knowledge).
3. Compare both and highlight differences.
4. Give the best final recommendation.

### Context:
{context}

### Question:
{user_input}
[/INST]
"""

    try:
        response = llm(prompt=prompt, max_tokens=550, temperature=0.3)
        return response["choices"][0]["text"]
    except Exception as e:
        return f"Error: {e}"


When to use this:

- When the context is incomplete or inconsistent
- When you want context-based + real-world guideline alignment
- For decisions like “Is this protocol correct?”

###Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(RAG4("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      35.51 ms /    61 runs   (    0.58 ms per token,  1717.73 tokens per second)
llama_print_timings: prompt eval time =   28575.60 ms /  3071 tokens (    9.30 ms per token,   107.47 tokens per second)
llama_print_timings:        eval time =   32629.71 ms /    60 runs   (  543.83 ms per token,     1.84 tokens per second)
llama_print_timings:       total time =   61586.84 ms /  3131 tokens


 I don't know. The provided text does not contain information about the protocol for managing sepsis in a critical care unit. It discusses the diagnosis and treatment of sepsis, but it does not provide a specific protocol for managing the condition in a critical care unit.


###Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(RAG4("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     282.22 ms /   493 runs   (    0.57 ms per token,  1746.87 tokens per second)
llama_print_timings: prompt eval time =   30292.18 ms /  3179 tokens (    9.53 ms per token,   104.94 tokens per second)
llama_print_timings:        eval time =  251749.98 ms /   492 runs   (  511.69 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  285040.55 ms /  3671 tokens


  Sure, I'll do my best to assist you! Here's my response based on the given context:

### Context:

• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate: In these cases, it usually lies behind the cecum or the ileum and mesentery of the right colon. A contraindication to appendectomy is inflammatory bowel disease involving the cecum. However, in cases of terminal ileitis and a normal cecum, the appendix should be removed.

### Question:
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Based on the given context, here's my answer:

Common symptoms for appendicitis include epigastric or periumbilical pain follow

###Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(RAG4("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     228.74 ms /   398 runs   (    0.57 ms per token,  1740.00 tokens per second)
llama_print_timings: prompt eval time =   33937.72 ms /  3450 tokens (    9.84 ms per token,   101.66 tokens per second)
llama_print_timings:        eval time =  206696.47 ms /   397 runs   (  520.65 ms per token,     1.92 tokens per second)
llama_print_timings:       total time =  243038.14 ms /  3847 tokens


  Based on the provided context, I understand that you are looking for information on sudden patchy hair loss, localized bald spots on the scalp, and possible causes and effective treatments or solutions. Here's my response:

Sudden patchy hair loss, also known as alopecia, can be caused by various factors such as hormonal imbalances, autoimmune disorders, infections, and stress. Localized bald spots on the scalp can be a symptom of these underlying conditions.

Possible causes of sudden patchy hair loss include:

1. Alopecia Areata: This is an autoimmune disorder that causes hair loss in patches. It can occur at any age and can affect anyone.
2. Telogen Effluvium: This is a condition where there is a sudden increase in hair shedding due to stress, hormonal changes, or other factors.
3. Folliculitis Decalvans: This is a condition where there is inflammation of the hair follicles, leading to scarring and hair loss.
4. Tinea Capitis: This is a fungal infection of the scalp that can cause

###Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(RAG4("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     254.00 ms /   441 runs   (    0.58 ms per token,  1736.24 tokens per second)
llama_print_timings: prompt eval time =   22693.78 ms /  2555 tokens (    8.88 ms per token,   112.59 tokens per second)
llama_print_timings:        eval time =  212792.45 ms /   440 runs   (  483.62 ms per token,     2.07 tokens per second)
llama_print_timings:       total time =  238079.44 ms /  2995 tokens


 As an AI language model, I can only provide information based on the provided context. Therefore, I cannot provide direct answers to your question as it is not within my programming to provide medical advice or engage in discussions that may require specialized knowledge or expertise. However, I can provide information based on the provided context, which may be helpful in answering your question.

Based on the provided context, traumatic brain injury (TBI) is a physical injury to brain tissue that temporarily or permanently impairs brain function. The treatment for TBI depends on the severity of the injury and may include supportive care, rehabilitation, and other interventions aimed at preventing complications and promoting recovery.

Supportive care may include measures to prevent systemic complications, provide good nutrition, prevent pressure ulcers, and provide physical therapy to prevent limb contractures. Rehabilitation services should be planned early and may include cognitiv

###Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(RAG4("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     318.93 ms /   550 runs   (    0.58 ms per token,  1724.52 tokens per second)
llama_print_timings: prompt eval time =   30600.54 ms /  3157 tokens (    9.69 ms per token,   103.17 tokens per second)
llama_print_timings:        eval time =  279859.86 ms /   549 runs   (  509.76 ms per token,     1.96 tokens per second)
llama_print_timings:       total time =  313884.48 ms /  3706 tokens


 ### Context:
batting, and layers 2 and 4 are elastic bandages. The injured limb is elevated above the heart for the first two days in a position that allows gravity to help drain edema fluid and thus minimize swelling. After 48 hours, periodic application of warmth (e.g., a heating pad) for 15 to 20 minutes may relieve pain and speed healing. Immobilization: Immobilization decreases pain and facilitates healing by preventing further injury and is helpful except for very rapidly healing injuries. Joints proximal and distal to the injury should be immobilized. A cast is usually used for fractures or other injuries that require weeks of immobilization. Rarely, swelling under a cast is severe enough to contribute to compartment syndrome (see p. 3213). Sometimes, if severe swelling is likely, a cast (and all padding) is cut open from end to end medially and laterally (bivalved). Patients with casts should be given written instructions to keep the cast dry, never to put an object inside the

## Question Answering using RAG - 5 - CONFIDENCE (Add confidence score and uncertainty handling)

- This prevents answers like “I don’t know” without explanation.

In [ ]:
def RAG5(user_input, llm):
    relevant_docs = retriever.get_relevant_documents(user_input)
    context = "\n".join([d.page_content.replace("\t", " ") for d in relevant_docs])

    prompt = f"""
[INST]
{qna_system_message}

Use the context below to answer.
If the context does not contain sufficient info, say:
"Information unclear in the provided context. Here is the general clinical guidance..."

At the end, output:
Confidence: High / Medium / Low

### Context:
{context}

### Question:
{user_input}
[/INST]
"""

    try:
        response = llm(prompt=prompt, max_tokens=450, temperature=0.3)
        return response["choices"][0]["text"]
    except Exception as e:
        return f"Error: {e}"


When to use this:

- When you need safety signals
- For real deployment in clinical assistance
- To detect hallucinations

###Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
print(RAG5("What is the protocol for managing sepsis in a critical care unit?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     253.79 ms /   450 runs   (    0.56 ms per token,  1773.13 tokens per second)
llama_print_timings: prompt eval time =   29612.51 ms /  3132 tokens (    9.45 ms per token,   105.77 tokens per second)
llama_print_timings:        eval time =  225512.72 ms /   449 runs   (  502.26 ms per token,     1.99 tokens per second)
llama_print_timings:       total time =  257821.54 ms /  3581 tokens


 I can provide information on the protocol for managing sepsis in a critical care unit based on the provided context. The context mentions that sepsis and septic shock are inflammatory states resulting from a systemic response to bacterial infection and that treatment includes aggressive fluid resuscitation, antibiotics, surgical excision of infected or necrotic tissues, and drainage of pus, supportive care, and sometimes intensive control of blood glucose and administration of corticosteroids and activated protein C.

Therefore, the protocol for managing sepsis in a critical care unit should include the following steps:

1. Early recognition of sepsis: The clinical team should be vigilant in identifying signs of sepsis, such as fever, hypotension, oliguria, and confusion, and should initiate treatment promptly.
2. Aggressive fluid resuscitation: The patient should receive intravenous fluids to raise mean arterial pressure (MAP) to at least 60 mmHg and maintain urine output of at least

###Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
print(RAG5("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     162.54 ms /   285 runs   (    0.57 ms per token,  1753.40 tokens per second)
llama_print_timings: prompt eval time =   30634.76 ms /  3180 tokens (    9.63 ms per token,   103.80 tokens per second)
llama_print_timings:        eval time =  142551.25 ms /   284 runs   (  501.94 ms per token,     1.99 tokens per second)
llama_print_timings:       total time =  174881.33 ms /  3464 tokens


 ### Context:

• Surgical removal • IV fluids and antibiotics • Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate.

### Question:
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Answer:

Symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, and pain increases with cough and motion. However, these classic symptoms appear in less than 50% of patients, and many variations of symptoms and signs occur.

 appendicitis cannot be cured via medicine, and surgical intervention is necessary. The surgical procedure for treating appendicitis is open or laparoscopic appendectomy. The choice of procedure de

###Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
print(RAG5("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     220.80 ms /   390 runs   (    0.57 ms per token,  1766.33 tokens per second)
llama_print_timings: prompt eval time =   34139.86 ms /  3450 tokens (    9.90 ms per token,   101.05 tokens per second)
llama_print_timings:        eval time =  199870.30 ms /   389 runs   (  513.81 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  236389.15 ms /  3839 tokens


  Based on the provided context, I can provide information on possible causes and effective treatments for sudden patchy hair loss, also known as localized bald spots on the scalp. Please note that I cannot provide personalized medical advice or diagnose individual cases.

Possible Causes:

1. Alopecia Areata: This is an autoimmune disorder that causes sudden hair loss in patches. It can occur at any age and affects both men and women.
2. Telogen Effluvium: This is a condition where there is a sudden increase in the number of hair follicles that stop growing and enter the resting phase, leading to excessive hair shedding. It can be triggered by physical or emotional stress, hormonal changes, or other factors.
3. Folliculitis Decalvans: This is a condition where the hair follicles become inflamed and damaged, leading to hair loss. It can be caused by bacterial or fungal infections, allergies, or other factors.
4. Nutritional Deficiencies: Deficiencies in essential nutrients like iron, z

###Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
print(RAG5("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     251.46 ms /   450 runs   (    0.56 ms per token,  1789.53 tokens per second)
llama_print_timings: prompt eval time =   22526.40 ms /  2555 tokens (    8.82 ms per token,   113.42 tokens per second)
llama_print_timings:        eval time =  215280.05 ms /   449 runs   (  479.47 ms per token,     2.09 tokens per second)
llama_print_timings:       total time =  240462.00 ms /  3004 tokens


 ### Context:
Chapter 324. Traumatic Brain Injury
Introduction
Traumatic brain injury (TBI) is physical injury to brain tissue that temporarily or permanently impairs brain function. Diagnosis is suspected clinically and confirmed by imaging (primarily CT). Initial treatment consists of ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery is often needed in patients with more severe injury to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation. In the US, as in much of the world, TBI is a common cause of death and disability. Causes include motor vehicle crashes and other transportation-related causes (eg, bicycle crashes, collisio

###Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
print(RAG5("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?" , lcpp_llm))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     254.99 ms /   450 runs   (    0.57 ms per token,  1764.75 tokens per second)
llama_print_timings: prompt eval time =   29722.30 ms /  3157 tokens (    9.41 ms per token,   106.22 tokens per second)
llama_print_timings:        eval time =  231236.05 ms /   449 runs   (  515.00 ms per token,     1.94 tokens per second)
llama_print_timings:       total time =  263768.18 ms /  3606 tokens


  ### Context:
batting, and layers 2 and 4 are elastic bandages. The injured limb is elevated above the heart for the first 2 days in a position that allows gravity to help drain edema fluid and thus minimize swelling. After 48 h, periodic application of warmth (eg, a heating pad) for 15 to 20 min may relieve pain and speed healing. Immobilization: Immobilization decreases pain and facilitates healing by preventing further injury and is helpful except for very rapidly healing injuries. Joints proximal and distal to the injury should be immobilized. A cast is usually used for fractures or other injuries that require weeks of immobilization. Rarely, swelling under a cast is severe enough to contribute to compartment syndrome (see p. 3213). Sometimes, if severe swelling is likely, a cast (and all padding) is cut open from end to end medially and laterally (bivalved). Patients with casts should be given written instructions: • To keep the cast dry • Never to put an object inside the cast •

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [ ]:
groundedness_rater_system_message  = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""

In [ ]:
relevance_rater_system_message = """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response_eval(user_input,answer,llm):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)


    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=500,
            temperature= 0,
            top_p= 0.95,
            repeat_penalty= 1.2,
            top_k= 50,
            stop=['INST'],
            echo=False
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens= 500,
            temperature= 0.4,
            top_p= 0.95,
            repeat_penalty= 1.2,
            top_k= 50,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
answer = RAG2(user_input, lcpp_llm)
groundedness_report, relevance_report = generate_ground_relevance_response_eval(user_input,answer,lcpp_llm)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     132.75 ms /   232 runs   (    0.57 ms per token,  1747.67 tokens per second)
llama_print_timings: prompt eval time =   34539.06 ms /  3339 tokens (   10.34 ms per token,    96.67 tokens per second)
llama_print_timings:        eval time =  114797.51 ms /   231 runs   (  496.96 ms per token,     2.01 tokens per second)
llama_print_timings:       total time =  150714.30 ms /  3570 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     139.76 ms /   251 runs   (    0.56 ms per token,  1795.96 tokens per second)
llama_print_timings: prompt eval time =   32607.91 ms /  3487 tokens (    9.35 ms per token,   106.94 tokens per second)
llama_print_timings:        eval time =  128151.80 ms /   250 runs   (  512.61 ms per token,     1.95 tokens per second)
llama_print_timings:       to

In [ ]:
print(groundedness_report, '\n\n', relevance_report)

  Based on the provided context, I would rate the answer as follows:

* 5 (fully derived) for items 1-4, as they are directly supported by the provided context.
* 4 (mostly derived) for item 5, as while the context mentions early administration of antibiotics effective against suspected pathogens, it does not explicitly state that surgical intervention should be performed.
* 3 (somewhat derived) for item 6, as while close monitoring of vital signs, laboratory values, and organ function is mentioned in the context, there is no direct mention of adjustments to therapy being made based on these assessments.
* 2 (limited) for item 7, as while protocol-driven management of sepsis is mentioned in the context, there is no direct mention of regular assessment of severity of illness and response to therapy, or escalation of care as indicated.

Overall, I would rate the answer as a 4 (mostly derived) as it accurately reflects the key elements of the protocol for managing sepsis in a critical car

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
answer = RAG2(user_input, lcpp_llm)
groundedness_report, relevance_report = generate_ground_relevance_response_eval(user_input,answer,lcpp_llm)
print(groundedness_report, '\n\n', relevance_report)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     225.59 ms /   400 runs   (    0.56 ms per token,  1773.13 tokens per second)
llama_print_timings: prompt eval time =   34399.36 ms /  3448 tokens (    9.98 ms per token,   100.23 tokens per second)
llama_print_timings:        eval time =  204759.41 ms /   399 runs   (  513.18 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  241525.03 ms /  3847 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      62.57 ms /   115 runs   (    0.54 ms per token,  1838.09 tokens per second)
llama_print_timings: prompt eval time =   36234.26 ms /  3764 tokens (    9.63 ms per token,   103.88 tokens per second)
llama_print_timings:        eval time =   57781.66 ms /   114 runs   (  506.86 ms per token,     1.97 tokens per second)
llama_print_timings:       to

  Based on the provided context, I would rate the answer as follows:

* Context: 5/5 (The context provides comprehensive information about the treatment of acute appendicitis, including surgical removal and antibiotics.)
* Answer: 4/5 (The answer provides accurate information about the treatment of acute appendicitis, but does not fully address the specific question asked.)

Overall rating: 4/5 (The answer is mostly complete but could benefit from more specific addressing of the question asked.) 

   Based on the provided context and question, I would rate the answer as follows:

Rating: 4 (Moderately Relevant)

The answer provides some relevant information about appendicitis and its treatment but does not fully address all aspects of the question. The answer mentions that surgical removal of the appendix is the treatment for appendicitis, but does not discuss other important aspects such as the symptoms of appendicitis, the diagnostic process, or alternative treatments for appendiciti

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
answer = RAG2(user_input, lcpp_llm)
groundedness_report, relevance_report = generate_ground_relevance_response_eval(user_input,answer,lcpp_llm)
print(groundedness_report, '\n\n', relevance_report)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      75.96 ms /   135 runs   (    0.56 ms per token,  1777.18 tokens per second)
llama_print_timings: prompt eval time =   35216.16 ms /  3718 tokens (    9.47 ms per token,   105.58 tokens per second)
llama_print_timings:        eval time =   68739.44 ms /   134 runs   (  512.98 ms per token,     1.95 tokens per second)
llama_print_timings:       total time =  104763.21 ms /  3852 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      92.93 ms /   169 runs   (    0.55 ms per token,  1818.59 tokens per second)
llama_print_timings: prompt eval time =   36463.99 ms /  3768 tokens (    9.68 ms per token,   103.33 tokens per second)
llama_print_timings:        eval time =   86774.84 ms /   168 runs   (  516.52 ms per token,     1.94 tokens per second)
llama_print_timings:       to

  Based on the provided context, I would rate the answer as follows:

* Content: 4/5 (The answer provides accurate information about the causes of sudden patchy hair loss and effective treatments)
* Contextual relevance: 4/5 (The answer is relevant to the question but does not fully address the specific context provided)
* Formatting: N/A (The answer is provided in paragraph form)

Overall, I would rate the answer as 4 out of 5. It provides accurate information but does not fully address the specific context provided. To improve the rating, the answer could be formatted in bullet points or numbered lists to make it easier to read and understand. Additionally, the answer could provide more specific examples or case studies to further illustrate the points being made. 

   Rating: 4 (Fully Relevant)

The provided answer fully covers all aspects of the question, addressing the possible causes of sudden patchy hair loss on the scalp, effective treatments or solutions, and the importance of

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
answer = RAG2(user_input, lcpp_llm)
groundedness_report, relevance_report = generate_ground_relevance_response_eval(user_input,answer,lcpp_llm)
print(groundedness_report, '\n\n', relevance_report)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      66.92 ms /   117 runs   (    0.57 ms per token,  1748.43 tokens per second)
llama_print_timings: prompt eval time =   26128.66 ms /  2823 tokens (    9.26 ms per token,   108.04 tokens per second)
llama_print_timings:        eval time =   55604.95 ms /   116 runs   (  479.35 ms per token,     2.09 tokens per second)
llama_print_timings:       total time =   82410.78 ms /  2939 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      83.62 ms /   154 runs   (    0.54 ms per token,  1841.64 tokens per second)
llama_print_timings: prompt eval time =   25262.18 ms /  2856 tokens (    8.85 ms per token,   113.05 tokens per second)
llama_print_timings:        eval time =   72918.81 ms /   153 runs   (  476.59 ms per token,     2.10 tokens per second)
llama_print_timings:       to

  Based on the provided context, I would rate the answer as follows:

The answer is derived from the context to a good extent (rating: 4). The response provides relevant information about treatments for brain injury, including supportive care, rehabilitation services, and neuroprotective agents. However, the answer could benefit from more specific details about the treatments and their prioritization based on individual needs. Additionally, while the answer mentions early intervention by rehabilitation specialists as crucial for maximizing functional recovery, it could provide more explanation about why this is important. Overall, the answer effectively summarizes key points related to treatments for brain injury but could benefit from more specific details and explanations. 

   I would rate the relevance of the answer as 4 out of 5. The answer provides a comprehensive overview of the treatments recommended for brain injury, including supportive care, rehabilitation services, and neur

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
answer = RAG2(user_input, lcpp_llm)
groundedness_report, relevance_report = generate_ground_relevance_response_eval(user_input,answer,lcpp_llm)
print(groundedness_report, '\n\n', relevance_report)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =     184.19 ms /   320 runs   (    0.58 ms per token,  1737.29 tokens per second)
llama_print_timings: prompt eval time =   32274.70 ms /  3425 tokens (    9.42 ms per token,   106.12 tokens per second)
llama_print_timings:        eval time =  160186.77 ms /   319 runs   (  502.15 ms per token,     1.99 tokens per second)
llama_print_timings:       total time =  194358.14 ms /  3744 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   11727.59 ms
llama_print_timings:      sample time =      85.06 ms /   154 runs   (    0.55 ms per token,  1810.51 tokens per second)
llama_print_timings: prompt eval time =   35619.89 ms /  3660 tokens (    9.73 ms per token,   102.75 tokens per second)
llama_print_timings:        eval time =   80735.27 ms /   153 runs   (  527.68 ms per token,     1.90 tokens per second)
llama_print_timings:       to

  Based on the information provided, I would rate the answer as follows:

The answer provides a comprehensive list of precautions and treatment steps for a person who has fractured their leg during a hiking trip. It covers essential aspects such as immobilization, pain management, elevation, cleanliness, monitoring, and follow-up care. Additionally, it acknowledges factors that may affect proper care and recovery, such as the type and severity of the fracture, patient's age and health status, location of medical facilities, and patient's ability to follow instructions. Overall, I would rate the answer as a 5 out of 5, indicating that it is completely derived from the context provided. 

   I would rate this answer as a 4 out of 5 in terms of relevance to the question asked. The answer provides a comprehensive list of precautions and treatment steps for a person who has fractured their leg during a hiking trip, including immobilization, pain management, elevation, cleanliness, monitorin

## Actionable Insights and Business Recommendations

###Actionable Insights

- The model sometimes over-answers beyond context: hallucination risk exists

  -> In Query 3 (hair loss), the answer declared “I don't know,” even though context did contain relevant medical information.

  -> In Query 1, the model added elements not fully present in the context.

Insight:
Your retrieval quality is good, but your prompt allows the model to supplement information beyond retrieved context, increasing hallucinations.

- Some queries require multi-hop medical reasoning: context alone not enough
  -> Appendicitis question requires: Symptoms, Whether medicine helps and Whether surgery required
- Relevance scores mostly 4–5: VectorDB search works, but context alignment needs tuning

- The LLM is sensitive to the prompt format → responses vary in structure
  -> Some answers come with: Bullet points, Paragraphs and Missing instruction compliance
- Model performance is highly dependent on chunk retrieval quality
  -> When relevant chunks include: Symptoms, Diagnosis steps and Treatment
  -> Model produces excellent answers (Query 5).

###Business Recommendations

- Introduce Medical-Domain Prompt Guardrails
  
  -> Do not add any information not found in context
  
  -> If context is insufficient, say so and request more details
  
  Business Value: Reduces hallucinations, increases trust and
- Improve Chunking and Retrieval Strategy

- Provide Answer Formatting and Consistency Standards

  -> Standardize output for medical QA: Diagnosis, Symptoms, Causes and Treatment

  Business Value: Professional, improves usability in clinical & tele-health applications

- Perform Query Analysis to Identify Missing Knowledge Areas

  -> Some questions (hair loss, brain injury) had weak retrieval because: They span dermatology and immunology or neurology and trauma

  Business Value: Faster, more accurate answers

- Maintain a Medical Safety Layer (High-Risk Domain Requirement)

  -> Provide only general information, not personalized diagnosis

  -> Do not give dosage without explicit context

  Business Value: Legal compliance, Risk mitigation and Safe deployment in healthcare settings

Final Summary

RAG system currently scores:
- Relevance: 4–5 (great)
- Groundedness: 3–4
- Completeness: 3–4


<font size=6 color='blue'>Power Ahead</font>
___